In [1]:
import cv2
import pandas as pd
import numpy as np
import os

In [25]:
#read the one image from Images folder
try:
    image_path = os.listdir('InputImage')[0]
except:
    print('There are no files in Images.')
image = cv2.imread('InputImage/'+image_path, 0)
image_copy = image.copy()

#Apply Gaussian blur to reduce noise-Used ChatGPT to do this
blurred_image = cv2.GaussianBlur(image, (5,5),0)

#Adjust contrast and brightness
processed_image = cv2.convertScaleAbs(blurred_image, alpha=1.5, beta=10)

#color-inversion
inverted_image = 255-processed_image
inverted_image[inverted_image < 150] = 0

True

In [26]:
# Find contours in the binary image
contours, _ = cv2.findContours(inverted_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [27]:
#Sort the contours based on x-values so the letters are in order
sorted_contours = sorted(contours, key=lambda contour: cv2.boundingRect(contour)[0])

In [28]:
#Use image copy instead of image so the contour lines are not printed
padding = 5  # You can change this value

for i, contour in enumerate(sorted_contours):
    # Filter out small contours that may be noise
    if cv2.contourArea(contour) > 200:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 1)
        
        # Add padding to the bounding box
        x -= padding
        y -= padding
        w += 2 * padding
        h += 2 * padding

        # Ensure the bounding box stays within the image boundaries
        x = max(0, x)
        y = max(0, y)
        w = min(image_copy.shape[1] - x, w)
        h = min(image_copy.shape[0] - y, h)
        
        # Extract the letter from the original image with padding
        letter = image_copy[y:y+h, x:x+w]
        
        #resize the images to 25x25 so there are 500 features
        letter = cv2.resize(letter, (25,25))
        
        # Save the letter as a separate image
        cv2.imwrite(f"IndividualLetters/letter_{i}.png", letter)
cv2.imwrite('ArchivedImages/image_with_contours.jpg', image)

True

In [24]:
#Clear folders: IndividualLetters, InputImage, ProcessedImage for next word
for f_name in os.listdir(path='InputImage'):
    os.remove("InputImage/" + f_name)
for f_name in os.listdir(path='ProcessedImage'):
    os.remove("ProcessedImage/" + f_name)
for f_name in os.listdir(path='IndividualLetters'):
    os.remove("IndividualLetters/" + f_name)

In [15]:
print(cv2.boundingRect(contours[0]))

(178, 84, 34, 41)
